 ## Exercise 7.1

 ### Question 1

 Which of the following is not a pre trained word embedding

 1. Glove
 2. Stanford
 3. Peeking
 4. All of the above

### Answer: 3 


### Question 2
 
 What should be the first argument to the Keras Embedding Layer.

1. The input vector dimensions
2. The output vector dimensions
3. The word embedding size
4. The vocabulary size

### Answer: 4

### Question 3
Which layer will you need to use if you want to directly connect the Embedding Layer with a Dense Layer

1. LSTM layer with return_sequence set to False
2. CNN layer with max pooling
3. Flatten layer
4. None of the Above

### Answer: 3


## Exercise 7.2

Using the IMDB dataset that we used for sentiment classification in section 7.4, perform sentiment classification using an RNN.
See if you can get better results with RNN as compared to the results in section 7.4.

### Solution

This is only a basic solution, you can add or remove RNN or DNN layers if you want along with the dropout sizes. 

In [0]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, LSTM, Dropout, Dense, Flatten, Input,  Embedding, Conv1D, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from google.colab import drive
drive.mount('/gdrive')
imdb_data= pd.read_csv("/gdrive/My Drive/datasets/IMDB Dataset.csv")

imdb_data.head()

imdb_data.sentiment.value_counts()
X = imdb_data["review"]

y = pd.get_dummies(imdb_data.sentiment, prefix='sent', drop_first=True).values
y.shape
def clean_text(doc):

    document = remove_tags(doc)

    document = re.sub('[^a-zA-Z]', ' ', document)

    document = re.sub(r"\s+[a-zA-Z]\s+", ' ', document)

    document = re.sub(r'\s+', ' ', document)

    return document
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(document):
    return TAG_RE.sub('', document)
X_sentences = []
reviews = list(X)
for rev in reviews:
    X_sentences.append(clean_text(rev))
X_train, X_test, y_train, y_test = train_test_split(X_sentences, y, test_size=0.20, random_state=42)
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
from numpy import array
from numpy import asarray
from numpy import zeros

embedd_dict= dict()
glove_embeddings = open('/gdrive/My Drive/datasets/glove.6B.100d.txt', encoding="utf8")

for embeddings in glove_embeddings:
    embedding_tokens = embeddings.split()
    emb_word = embedding_tokens [0]
    emb_vector = asarray(   embedding_tokens[1:], dtype='float32')
    embedd_dict [emb_word] = emb_vector 

glove_embeddings.close()
embedd_mat= zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embedd_dict.get(word)
    if embedding_vector is not None:
        embedd_mat[index] = embedding_vector
embedd_mat.shape


embedding_inputs = Input(shape=(maxlen))
embedding_layer = Embedding(vocab_size, 100, weights=[embedd_mat], trainable=False)(embedding_inputs )
lstm1 = LSTM(128, activation='relu', return_sequences=True)(embedding_layer)

lstm2 = LSTM(64, activation='relu', )(lstm1)

drop1 = Dropout(0.2)(lstm2)
dense1 = Dense(512, activation = 'relu')(drop1)
drop2  = Dropout(0.2)(dense1)
output_layer = Dense(1, activation= 'sigmoid')(drop2)

model = Model(embedding_inputs, output_layer )
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot1.png', show_shapes=True, show_layer_names=True)
history = model.fit(X_train, y_train, batch_size= 128, epochs=10, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)
print(score[0])
print(score[1])

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Epoch 1/10
250/250 [==============================] - 112s 449ms/step - loss: 19787.7559 - accuracy: 0.5923 - val_loss: 0.6327 - val_accuracy: 0.6505
Epoch 2/10
250/250 [==============================] - 112s 448ms/step - loss: 0.6441 - accuracy: 0.6342 - val_loss: 0.6369 - val_accuracy: 0.6400
Epoch 3/10
250/250 [==============================] - 113s 450ms/step - loss: 0.6387 - accuracy: 0.6330 - val_loss: 0.6268 - val_accuracy: 0.6503
Epoch 4/10
250/250 [==============================] - 115s 461ms/step - loss: 0.6338 - accuracy: 0.6378 - val_loss: 0.6219 - val_accuracy: 0.6509
Epoch 5/10
164/250 [==================>...........] - ETA: 35s - loss: 0.6314 - accuracy: 0.6399